## The purpose of this project is to implement a BERT-based model for sentiment binary classification on the Twitter US Airlines Sentiment dataset. ##
The key columns are:  
* text: The content of the tweets
* airline_sentiment: The sentiment labels for prediction

In [23]:
# Import required libraries 
import torch # for deep learning
import numpy as np # for numerical operations
import torch.nn as nn  # Neural network 
import torch.optim as optim  # Optimizers for training
from transformers import BertTokenizer, BertModel  # Pre-trained BERT tokenizer and model
from sklearn.model_selection import train_test_split  # Split data into training, validation, and test sets
import time  # To measure training time
from torch.utils.data import DataLoader, Dataset  # Classes for handling datasets and batching
import pandas as pd


In [37]:
# Load the dataset 
def load_dataset():
    df = pd.read_csv("Tweets.csv")
    texts = df['text'].tolist()
    sentiment_map = {'positive': 2, 'neutral': 0, 'negative': 1}
    labels = dataset['airline_sentiment'].map(sentiment_map).tolist()
    return texts, labels

In [39]:
# Set random seed for reproducibility 
SEED = 1234
torch.manual_seed(SEED)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128


# Define dataset class for text classification
class AirlineSentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Tokenize the text
        tokens = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
        'input_ids': tokens['input_ids'].flatten(), 
        'attention_mask': tokens['attention_mask'].flatten(), 
        'label': torch.tensor(label, dtype=torch.long)
    }

# Extract key columns 
texts, labels = load_dataset()

# Split the dataset into test and training 
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.4, random_state=SEED)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.5, random_state=SEED)

# Create dataset objects for each split 
train_dataset = AirlineSentimentDataset(train_texts, train_labels, tokenizer, max_length)
valid_dataset = AirlineSentimentDataset(valid_texts, valid_labels, tokenizer, max_length)
test_dataset = AirlineSentimentDataset(test_texts, test_labels, tokenizer, max_length)

# Set batch size
BATCH_SIZE = 16

# Create DataLoader to batch the data 
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define the BERT-GRU model
class BERTGRUSentiment(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, dropout):
        super().__init__()
        self.bert = bert  # Pre-trained BERT model
        self.rnn = nn.GRU(bert.config.hidden_size, hidden_dim, batch_first=True, bidirectional=True)  # GRU layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Fully connected layer for binary classification
        self.dropout = nn.Dropout(dropout)  # Dropout for regularization

    def forward(self, input_ids, attention_mask):
        # Pass input through BERT and extract embeddings
        with torch.no_grad():  # Freeze BERT weights
            embedded = self.bert(input_ids, attention_mask=attention_mask).last_hidden_state
        # Pass embeddings through GRU
        _, hidden = self.rnn(embedded)
        # Concatenate the last forward and backward GRU hidden states
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        return self.fc(hidden)  # Pass through the fully connected layer

# Initialize the BERT-GRU model
bert = BertModel.from_pretrained('bert-base-uncased')
model = BERTGRUSentiment(bert, hidden_dim=128, output_dim=3, dropout=0.3).to(torch.device("cpu"))

# Freeze BERT parameters to avoid updating them during training
for param in model.bert.parameters():
    param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss() # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=2e-5)  # Adam optimizer

# Calculate accuracy
def calculate_accuracy(preds, y):
    _, predicted = torch.max(preds, 1)
    correct = (predicted == y).float()
    return correct.sum() / len(correct)

# Function to train the model
def train(model, loader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    
    for batch in loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        predictions = model(input_ids, attention_mask)
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

# Function to evaluate the model
def evaluate(model, loader, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['label']
            
            predictions = model(input_ids, attention_mask)
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss / len(loader), epoch_acc / len(loader)

# Training loop
N_EPOCHS = 4
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS + 1):
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    end_time = time.time()

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'bert_gru_model.pt')

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Valid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc*100:.2f}% | Time: {end_time - start_time:.2f}s")

# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('bert_gru_model.pt'))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%")

# Prediction function for multi-class classification
def predict_sentiment(model, tokenizer, text):
    tokens = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    model.eval()
    
    with torch.no_grad():
        prediction = model(input_ids, attention_mask)
        _, predicted_class = torch.max(prediction, 1)
        probabilities = torch.softmax(prediction, dim=1)
    
    sentiment_classes = {0: "Negative", 1: "Neutral", 2: "Positive"}
    return sentiment_classes[predicted_class.item()], probabilities[0].tolist()

# Example predictions
print(predict_sentiment(model, tokenizer, "VirginAmerica The flight was amazing, thank you!"))
print(predict_sentiment(model, tokenizer, "AmericanAir worst customer service ever. Never flying with you again."))
print(predict_sentiment(model, tokenizer, "SouthwestAir Flight delayed but staff was helpful."))

Epoch 1 | Train Loss: 0.916 | Train Acc: 59.91% | Valid Loss: 0.835 | Valid Acc: 63.61% | Time: 428.92s
Epoch 2 | Train Loss: 0.845 | Train Acc: 61.61% | Valid Loss: 0.774 | Valid Acc: 63.80% | Time: 422.78s
Epoch 3 | Train Loss: 0.780 | Train Acc: 64.43% | Valid Loss: 0.713 | Valid Acc: 68.61% | Time: 419.73s
Epoch 4 | Train Loss: 0.722 | Train Acc: 68.86% | Valid Loss: 0.649 | Valid Acc: 73.75% | Time: 421.09s
Epoch 5 | Train Loss: 0.663 | Train Acc: 72.77% | Valid Loss: 0.595 | Valid Acc: 75.16% | Time: 420.42s
Test Loss: 0.599 | Test Acc: 75.53%
('Positive', [0.14642806351184845, 0.05673369765281677, 0.7968382835388184])
('Neutral', [0.12194711714982986, 0.8108943700790405, 0.06715847551822662])
('Neutral', [0.26911893486976624, 0.6571492552757263, 0.07373186200857162])
